# Import packages

In [2]:
import os
import pandas as pd
import json
from google.colab import drive

# Load data

In [3]:
drive.mount('/content/gdrive')
!dir './gdrive/Shareddrives/twlim_ucsd_drive/Data'

Mounted at /content/gdrive
cve.json


In [4]:
file_path = './gdrive/Shareddrives/twlim_ucsd_drive/Data/cve.json'
with open(file_path, 'r') as fp:
    data = json.load(fp) 
len(data)

150600

In [51]:
cve_id = list()
last_modified_date = list()
published_date = list()
attack_vector = list()
attack_complexity = list()
privileges_required = list()
user_interaction = list()
scope = list()
confidentiality = list()
integrity = list()
availability = list()
description = list()

for idx in range(len(data)):
    try:
        if data[idx].get('impact') and data[idx]['impact'].get('baseMetricV3'):
            cve_id.append(data[idx]['cve']['CVE_data_meta']['ID'])
            attack_vector.append(data[idx]['impact']['baseMetricV3']['cvssV3']['attackVector'])
            attack_complexity.append(data[idx]['impact']['baseMetricV3']['cvssV3']['attackComplexity'])
            privileges_required.append(data[idx]['impact']['baseMetricV3']['cvssV3']['privilegesRequired'])
            user_interaction.append(data[idx]['impact']['baseMetricV3']['cvssV3']['userInteraction'])
            scope.append(data[idx]['impact']['baseMetricV3']['cvssV3']['scope'])
            confidentiality.append(data[idx]['impact']['baseMetricV3']['cvssV3']['confidentialityImpact'])
            integrity.append(data[idx]['impact']['baseMetricV3']['cvssV3']['integrityImpact'])
            availability.append(data[idx]['impact']['baseMetricV3']['cvssV3']['availabilityImpact'])
            description.append(' '.join([text['value'] for text in data[idx]['cve']['description']['description_data']]))
            last_modified_date.append(data[idx]['lastModifiedDate'])
            published_date.append(data[idx]['publishedDate'])
    except KeyError:
        print('Key error at index: {}'.format(idx))
        break

df = pd.DataFrame({'cve_id': cve_id,
                   'attack_vector': attack_vector, 
                   'attack_complexity': attack_complexity,
                   'privileges_required': privileges_required,
                   'user_interaction': user_interaction,
                   'scope': scope,
                   'confidentiality': confidentiality,
                   'integrity': integrity,
                   'availability': availability,
                   'description': description,
                   'last_modified_date': last_modified_date,
                   'published_date': published_date,
                   })
print('Data size: {}'.format(len(df)))

Data size: 77020


# Train test split

In [47]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.1, random_state=88)
len(df_train), len(df_test)

(69318, 7702)

In [48]:
df_test.sample(3)

,cve_id,attack_vector,attack_complexity,privileges_required,user_interaction,scope,confidentiality,integrity,availability,description,last_modified_date,published_date
68552,CVE-2016-3527,NETWORK,LOW,NONE,NONE,UNCHANGED,HIGH,HIGH,NONE,Unspecified vulnerability in the Oracle Demand...,2017-09-01T01:29Z,2016-07-21T10:13Z
45762,CVE-2018-3287,LOCAL,LOW,NONE,REQUIRED,CHANGED,HIGH,HIGH,HIGH,Vulnerability in the Oracle VM VirtualBox comp...,2019-10-03T00:03Z,2018-10-17T01:31Z
25288,CVE-2020-12474,NETWORK,LOW,NONE,REQUIRED,UNCHANGED,HIGH,NONE,NONE,"Telegram Desktop through 2.0.1, Telegram throu...",2020-05-06T19:56Z,2020-05-01T14:15Z


# Save the split train test data

In [54]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()
output_dir = './gdrive/Shareddrives/twlim_ucsd_drive/Data'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print('Saving data to %s' % output_dir)

# Save output csv
df_train.to_csv(os.path.join(output_dir, r'cve_train.csv'), header=True)
df_test.to_csv(os.path.join(output_dir, r'cve_test.csv'), header=True)

Saving data to ./gdrive/Shareddrives/twlim_ucsd_drive/Data
